#### 데이터 불러오기


In [2]:
import pandas as pd

df = pd.read_csv('./Data/train.csv')
df.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


#### 결측치 확인
NA: Not Available 값이 누락 된 값
<br><span style="color: gray">아니 왜 isna()안쓰고 함수정의하냐고
<br>아니 심지어 함수 안에서 isna()쓰는데???
</span>

In [5]:
def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col}입니다.')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다.')
    return missing_col

missing_col = check_missing_col(df)

결측치가 존재하지 않습니다.


이제 본격적으로 모델을 설계하기 위해 데이터를 문서와 label로 나누어 줌

In [9]:
x = df.document # train 데이터에서 문서 추출
y = df.label # train 데이터에서 라벨 추출

x.head()
y.head()

0    0
1    1
2    1
3    0
4    0
Name: label, dtype: int64

#### CountVectorizer 의 이해
컴퓨터는 문장(텍스트)을 계산할 수 없다.<br>
컴퓨터가 계산을 하기 위해서는 먼저 문장을 숫자로 바꾼 다음 계산하도록 해야 한다.<br>
이것을 워드 임베딩(word embedding)이라고 한다.<br>
이번에는 CountVectorizer 기법을 사용해 문장을 숫자로 표현해 본다.<br>
CountVectorizer는 입력된 문장을 토큰화(Tokenize)하여 토큰의 등장 빈도 벡터로 바꿔주는 기법<br>
<span style="color: gray">벡터가 뭔데...</span><br>
토큰화란 문장을 의미가 있는 최소의 단위로 쪼개는 것을 의미함<br>
예를 들어 "hello, I am a data scientist!" 라는 문장을 띄어쓰기 기준으로 토큰화 시키면<br>
"hello", ",", "I", "am", "a", "data", "scientist", "!" 라는 8개의 토큰으로 토큰화 된다.<br>
토큰화를 하는 방식은 여러가지가 있지만 이번에는 띄어쓰기를 기준으로 토큰화를 진행한다.

CounterVectorizer 사용을 위해 sklearn 패키지에 있는 CounterVectorizer 불러옴

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

sample_vectorizer = CountVectorizer() # 객체 생성

이제 CounterVectorizer를 학습 시켜주어야 함<br>
어떠한 단어들을 사용할 지, 어떠한 단어가 중요도를 가지는지 학습하는 과정임<br>
CounterVectorizer의 경우 단어의 출현 빈도를 기준으로 문장을 벡터(=숫자의 집합)<span  style="color: gray">아 벡터가 숫자의 집합을 말하는구나</span>로 바꿔줍니다.

In [18]:
sample_text1 = ['hello, my name is dacon and I am a data scientist!']

sample_vectorizer.fit(sample_text1) # CountVectorizer 학습

CountVectorizer()

학습이 완료됨<br>
먼저 sample_vectorizer가 학습한 단어 목록을 확인해봄<br>
이를 Vocabulary라고 하며, 앞으로 sample_vectorizer는 이 Vocab을 기준으로 새로운 문장을 숫자로 바꿔줄 것

In [19]:
print(sample_vectorizer.vocabulary_) # Vocabulary

{'hello': 4, 'my': 6, 'name': 7, 'is': 5, 'dacon': 2, 'and': 1, 'am': 0, 'data': 3, 'scientist': 8}


"hello, my name is Kim Young Ho and I am a practice data analysis" 문장으로 vectorizer를 학습시켰다.<br>
sklearn의 CounterVector는 "I", "a"와 같이 길이가 2 이하인 단어들과 ",", "!" 같은 특수 문자를 읨가 적다고 판단하여 제외시킨다.<br>
그래서 최종적으로 9개의 단어들이 Vocab에 포함되어있다.<br>
그럼 이제 다른 문장을 sample_vectorizer를 활용해 백터로 바꿔본다

In [20]:
sample_text2 = ["you are learning dacon data science"]

sample_vector = sample_vectorizer.transform(sample_text2)
print(sample_vector.toarray())

[[0 0 1 1 0 0 0 0 0]]


"you are learning dacon data science" 문장을 sample_vectorizer 를 활용해 transform 해보았다.<br>
단어들의 출현 빈도로 이루어진 크기 9의 벡터가 출력되었다.<br>
이 벡터를 BoW라고 한다.<br>
BoW란 "Bag of Words"의 약자로 CounterVectorizer로 변환된 단어의 집합을 말한다.<br>
sample_text2 문장 내에는 "dacon", "data" 단어가 각 1번 씩 출현했다.<br>
따라서 "dacon"과 "data"에 해당하는 인덱스의 값이 1로 표시되어 있고, 등장하지 않은 단어들은 0으로 표시되어 있다.

In [21]:
sample_text3 = ['you are learning dacon data science with movie data']

sample_vector2 = sample_vectorizer.transform(sample_text3)
print(sample_vector2.toarray())

[[0 0 1 2 0 0 0 0 0]]


"you are learning dacon data science with movie data" 문장에는 "dacon" 단어가 1번, "data" 단어가 2번 출현했다.<br>
따라서 "dacon"단어에 해당하는 인덱스에는 1의 값이, "data"에 해당하는 인덱스에는 2의 값이 할당되었다.<br>
CountVectorizer 는 위와 같이 작동함<br>
이해를 위해 CountVectorizer를 하나의 문장을 사용해 학습 시켰기에 생성된 Vocab과 BoW의 크기는 모두 9였다.<br>
하지만 더 큰 데이터를 사용해 CountVectorizer를 학습시킨다면 Vocab과 BoW의 크기는 더욱 증가한다.

#### 영화 리뷰 데이터
먼저 새로운 객체를 생성하여 CountVectorizer를 할당해 준다.<br>
다음 미리 준비해두었던 데이터를 사용해 CountVectorizer를 학습 시켜준다.

In [22]:
vectorizer = CountVectorizer() # countvectorizer 생성
vectorizer.fit(x) # countvectorizer 학습
x = vectorizer.transform(x) # transform

이제 CountVectorizer를 학습시켰다.<br>
x는 이제 자연어가 아닌 숫자, 벡터 데이터로 이루어져 있다.<br>
이제는 머신러닝 모델의 input 값으로 적절한 데이터의 형식을 갖춘 것이다.

#### Logistic Regression
분류 문제에 많이 사용되는 LogisticRegression 모델을 사용하여 리뷰들의 감정을 분류해 보았다.

In [23]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression() # 객체에 모델 할당
model.fit(x, y) # 모델 학습 

LogisticRegression()

모델 학습이 완료되었다.<br>
모델이 잘 학습되었는지 직접 문장을 입력해서 확인해보자

In [31]:
# run model
x_pred = vectorizer.transform(['영화 완전 꿀잼'])
y_pred = model.predict(x_pred)
print(y_pred)

[1]
